# Scraping articles from Inquirer.com 
### By: Kenneth Loquinte

In [1]:
import requests
import time
from datetime import datetime
from bs4 import BeautifulSoup

page_counter = 1
URL="https://newsinfo.inquirer.net/category/latest-stories/page/"
headers = {'User-Agent': '...'} #site does not seem to accept requests without headers 

## Checking if we have access to URL

In [2]:
page=requests.get(URL+str(page_counter), headers=headers)

In [3]:
page.content

b'\r\n<!DOCTYPE html>\r\n<html lang="en">\r\n<head>\r\n\t<meta charset="UTF-8">\r\n\t\r\n\t\r\n     \t\r\n\t<meta property="fb:app_id" content="175228195865217" />\r\n\t\r\n\t<meta property="fb:pages" content="208783679172573" />\t\r\n\t<meta property="og:ttl" content="345600" />\r\n\t<meta property="og:title" content="| The latest from Inquirer News" /> \r\n\t<meta property="og:description" content="Latest Philippine News for Filipinos" />\r\n\t\t\t<meta property="og:image" content="//www.inquirer.net/wp-content/uploads/2016/04/inqdefimg.jpg" />\r\n\t\t\t<meta name="medium" content="news" />\t\r\n\t<meta name="copyright" content="Copyright (c) 2001-2021 INQUIRER.net" />\r\n\t\r\n\t<meta name="google-site-verification" content="IrkUmIvfUjRBrt0o90DcVDSkbDM8UjMFDCIvMk_Kxso" />\r\n\t\r\n\t\t\r\n\t\t<meta name=\'dailymotion-domain-verification\' content=\'dmicdjab7p8l3v93l\' />\r\n\r\n\t\t\r\n\t\r\n\t<meta name="google-site-verification" content="iclsJL576k2YzKuw4vqKeesL9H1KkYks62rB5C4mU3k

## Finding the correct page based on date

In [4]:
START_DATE = "March 12, 2021"
STOP_DATE = "March 10, 2021"

page_counter = 42

def check_if_date_match(match_date, dates):
    global page_counter
    for date in dates:
        if any(match_date in s for s in date):
            return True
        else:
            return False
        
def get_new_page(page_num):
    page=requests.get(URL+str(page_counter), headers=headers)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

scrape = False


# go to correct start page
while not scrape:
    soup = get_new_page(page_counter)
    page_counter+=1
    article_list=soup.find(id='inq-channel-left')
    dates = article_list.find_all(id='ch-postdate')
    print(dates)
#     time.sleep(5)
    scrape = check_if_date_match(START_DATE, dates)    

[<div id="ch-postdate"><span>March 14, 2021</span> <span>BY:  Katrina Hallare</span></div>, <div id="ch-postdate"><span>March 14, 2021</span> <span></span></div>, <div id="ch-postdate"><span>March 14, 2021</span> <span></span></div>, <div id="ch-postdate"><span>March 14, 2021</span> <span></span></div>, <div id="ch-postdate"><span>March 14, 2021</span> <span></span></div>, <div id="ch-postdate"><span>March 14, 2021</span> <span></span></div>, <div id="ch-postdate"><span>March 14, 2021</span> <span>BY:  Armand Galang</span></div>, <div id="ch-postdate"><span>March 14, 2021</span> <span>BY:  Katrina Hallare</span></div>, <div id="ch-postdate"><span>March 14, 2021</span> <span>BY:  Katrina Hallare</span></div>, <div id="ch-postdate"><span>March 14, 2021</span> <span>BY:  Romar Miranda</span></div>]
[<div id="ch-postdate"><span>March 14, 2021</span> <span>BY:  Katrina Hallare</span></div>, <div id="ch-postdate"><span>March 14, 2021</span> <span>BY:  Delfin T. Mallari Jr.</span></div>, <div

## Get list of all article links

In [5]:
article_links = []
while scrape:
    print(page_counter)
    soup = get_new_page(page_counter)
    page_counter+=1
    article_list=soup.find(id='inq-channel-left')
    articles = article_list.find(id='ch-ls-box')
    dates = article_list.find_all(id='ch-postdate')

    for element in articles.next_siblings:
        if(element != '\n' #avoiding NavigableString type
           and element.get('id') #making sure element has id
           and element["id"] == 'ch-ls-box' #only getting article divs (exception is the ad box)
          ): 
            if (element.a):
                article_links.append(element.a["href"])
    
    found_end = check_if_date_match(STOP_DATE, dates)
    scrape = not found_end
    

50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69


## Getting content of each article

In [8]:
import urllib.request #use urrlib.request instead because requests module does not give full html of article page

def get_page(url):
    req = urllib.request.Request(url,headers=headers)
    page= urllib.request.urlopen(req).read()
    soup = BeautifulSoup(page, 'html.parser')
    return soup
    

news_json = []

    
for link in article_links:
    print(link)
    soup = get_page(link)
    try:
        article_date = soup.find(id="art_plat")
        article_title = soup.find('h1', class_="entry-title")
        article_author = soup.find(id='art_author')
        article_body = soup.find(id="article_content")


        author = article_author.span.a.string.strip()
        title = article_title.string.strip()

        if len(article_date.contents) > 1:
            date = article_date.contents[2].strip(" /")

        else:
            date = article_date.string.strip()
            
        if any(STOP_DATE in s for s in date):
            break


        full_article = ""


        for element in article_body.contents[1].children:
            if (element.name == "p" and element.string):
                full_article += element.string.strip() + " "


        print(title)
        print(date)    
        print(author)

        print("")
        print(full_article)
        
        news_json.append({
                "title": title,
                "author": author,
                "date": date,
                "full_article": full_article,
            })
        
    except:
        print("Cannot get data...moving to next article")
        
print("Finished scraping.")

https://newsinfo.inquirer.net/1406307/bohol-gov-tells-panglao-mayor-stop-resort-plan-to-hold-foam-party
Bohol gov tells Panglao mayor: Stop resort plan to hold foam party
08:53 PM March 12, 2021
Leo Udtohan

TAGBILARAN CITY—The provincial government on Friday (March 12) directed the mayor of the tourist town of Panglao to order a resort there to drop plans to hold a foam party in which participants dance while surrounded by suds or bubbles. The governor said guidelines of the Inter-Agency Task Force for Emerging Diseases (IATF) allowed only hotels and other tourist establishments with DOT accreditation to operate during the state of public health emergency. The governor said the order was being given “to avoid the recurrence of reported violations of minimum health and safety protocols” to prevent the transmission of SARS Cov2, the coronavirus that causes COVID-19. The order, Yap said, was for the mayor’s “strict compliance.” 
https://newsinfo.inquirer.net/1406301/texas-attorney-genera

‘Glimmer of hope’: Army division in Samar starts vaccination
08:06 PM March 12, 2021
Joey Gabieta

TACLOBAN CITY—At least 600 soldiers and medical workers of the Army’s 8th Infantry Division based in Catbalogan City, Samar province had been injected with CoronaVac, the coronavirus vaccine manufactured by Chinese company Sinovac, starting on Thursday (March 11). Lt. Col. Christopher Dayag, head of Camp Lukban Station Hospital inside the division grounds, said 100 health workers and soldiers got the first doses of the vaccine. The 8th Infantry Division received a total of 12,000 doses of CoronaVac from the Army national headquarters. “Many lives have been lost,” he said. “Here in 8th Infantry Division, one of our Stormtroopers perished after acquiring the vaccine,” Dinoso said. “With the vaccine around, it gives us a glimmer of hope that we can return to our normal day-to-day lives,” he said in a statement. “Soldiers’ vaccination is not an option. It is our duty to receive the vaccine,” 

Cannot get data...moving to next article
https://newsinfo.inquirer.net/1406237/south-africa-could-get-20-million-more-jj-vaccine-doses-says-minister
Cannot get data...moving to next article
https://newsinfo.inquirer.net/1406205/fake-id-with-president-dutertes-photo-seized-by-police-in-california
Fake ID with President Duterte’s photo seized by police in California
05:13 PM March 12, 2021
Ian Biong

A fake driver’s license bearing President Rodrigo Duterte’s photo was seized recently by the Los Angeles Police Department (LAPD) in California, USA. “Good thing our Officers patrol Historic Filipino Town and are up to speed. Nice try,” police quipped. The owner of the fake California driver’s license, who had a Glendale address, used a smirking photo of the Philippine president. Although police censored some information printed on the ID, including the name, the signature beneath the ID photo seemingly read “Norman D.” Police did not disclose if they were able to identify the owner of the f

‘Back to square one’: Senators lament slow gov’t response a year into pandemic
04:11 PM March 12, 2021
Christia Marie Ramos

MANILA, Philippines — One year since the pandemic started, at least two senators on Friday lamented that the country seems to be “back to square one” in terms of government response, amid the spike in recorded COVID-19 cases in the Philippines. “Today feels like March 2020 all over again, with the spike in COVID cases and the slow vaccine roll-out,” the senator added. “Mag-iisang taon na ang kung ano-anong CQ (community quarantine) o lockdown,”Pangilinan said, citing the importance of mass testing, contact tracing and isolation. (It’s been one year since we’ve imposed all sorts of community quarantine or lockdown.) Meanwhile, Senator Joel Villanueva said that a year into the pandemic, the country has yet to see the big leap to curb the coronavirus. “It’s so disappointing to note na it’s been a year and we could hardly see yung malaking paglukso o pagtalon natin p

Cannot get data...moving to next article
https://newsinfo.inquirer.net/1406176/sea-slugs-lose-heads-to-rid-bodies-of-parasites-japan-researchers-show
Cannot get data...moving to next article
https://newsinfo.inquirer.net/1406173/japan-vaccine-chief-says-to-use-six-dose-syringes-from-mid-april
Cannot get data...moving to next article
https://newsinfo.inquirer.net/1406163/more-protests-held-in-myanmar-as-suu-kyis-lawyer-dismisses-bribery-claims
Cannot get data...moving to next article
https://newsinfo.inquirer.net/1406166/doh-4-regions-face-difficulty-in-submitting-samples-for-genome-sequencing
Cannot get data...moving to next article
https://newsinfo.inquirer.net/1406129/push-for-duterte-vice-presidency-still-just-a-draft-says-house-leader
Push for Duterte vice presidency ‘just a draft,’ says House leader
02:18 PM March 12, 2021
Neil Arwin Mercado

MANILA, Philippines — A lawmaker who is a member of the ruling PDP-Laban party on Friday said that a resolution calling for President Rodrig

...

Cop hurt, suspect killed in Camarines Sur town
12:10 PM March 11, 2021
Ma. April Mier-Manjares

LEGAZPI CITY –– A police officer was hacked while the suspect, who created the public disturbance, was shot dead in the town of Calabanga, Camarines Sur, on Wednesday night. While the policemen were negotiating for his surrender, Catimbang hacked Staff Sergeant Russel Pron on his right arm. Catimbang died on the spot while Pron was brought to a hospital for treatment. Dacusin was unharmed. Averilla said Catimbang was believed to be mentally challenged. 
https://newsinfo.inquirer.net/1405614/religious-leaders-alarmed-at-violence-fast-spreading-like-covid-19-tells-security-forces-to-respect-human-rights
Religious leaders alarmed at violence ‘fast spreading like COVID-19’; tells security forces to respect human rights
11:57 AM March 11, 2021
Ryan D. Rosauro

ILIGAN CITY –– A group of religious leaders from various denominations in the country has expressed alarm over the recent spate of killing

Cannot get data...moving to next article
https://newsinfo.inquirer.net/1405571/3000-nurses-dead-covid-19-exodus-looming-global-federation
Cannot get data...moving to next article
https://newsinfo.inquirer.net/1405568/pagasa-cloudy-thursday-with-isolated-rain-showers-as-lpa-nears-mindanao
Pagasa: Cloudy Thursday with isolated rain showers as LPA nears Mindanao
07:39 AM March 11, 2021
Frances Mangosing

The LPA was last spotted 465 kilometers east of General Santos City, the Philippine Atmospheric Geophysical and Astronomical Services Administration said. Metro Manila and the rest of Luzon will have partly cloudy to cloudy skies with isolated light rains. The northern and eastern sections of Luzon, and the eastern section of Visayas will have moderate to strong winds moving northeast, while coastal waters will be moderate to rough. The rest of the country will have light to moderate winds moving in the east to northeast direction. Coastal waters will be slight to moderate, Pagasa said. 


PopCom wants special protection for women amid pandemic
10:02 PM March 10, 2021
Gabriel Pabico Lalu

MANILA, Philippines —With more than one-fourth of the population expressing concern about the reported domestic violence during the COVID-19 pandemic, the Commission on Population and Development (PopCom) has stressed the importance of ensuring women’s safety. The SWS survey, released last February 2021, showed that 11 percent of the 1,500 adults interviewed said that physical violence is the top problem of women, with seven percent each citing sexual and emotional violence. “Recently, POPCOM disclosed the results of a recent Social Weather Stations survey, which revealed that 1 out of 4, or 25%, of Filipino adults across the Philippines, cited harmful acts in various forms as among the most pressing problems of women during the present health crisis,” it added. After the COVID-19 pandemic forced the authorities to place Luzon and many other areas under strict lockdown, officials and wo

SC suspends work to give way for ‘thorough’ building disinfection
07:30 PM March 10, 2021
Tetch Torres-Tupas

MANILA, Philippines—The Supreme Court has suspended work from March 11 to March 14, 2021 to give way for disinfection of offices and buildings due to the rising number of COVID-19 infections. During this period of disinfection and sanitation, Peralta said they would allow committees with previously scheduled meetings to proceed, depending on the discretion of the chairperson or head of the committee. They also expect on-duty personnel of the Medical and Dental Services, Security and Maintenance Divisions, Office of Administrative Services-Supreme Court to report for work. Currently, the country’s active COVID-19 case count went up to 44,470 with 2,886 new infections. The Supreme Court last Tuesday has also suspended the oral arguments on the petitions against the Anti-Terrorism Act of 2020 due to the need for the justices to undergo self-quarantine. 
https://newsinfo.inquirer.n

Light to moderate rains over Visayas, Mindanao due to LPA — Pagasa
06:01 PM March 10, 2021
Gabriel Pabico Lalu

MANILA, Philippines — Light to moderate rains may be experienced in different areas of Visayas and Mindanao due to the low-pressure area (LPA) spotted east of General Santos City, state meteorologists said Wednesday. (The low pressure area still has a low chance of developing into a tropical depression.  However, it would still cause light to moderate to at times heavy rains over the whole Visayas and Mindanao.) For the remaining parts of Luzon, warm and humid weather is expected to persist. Temperatures in Luzon for Thursday would be lower due to the presence of the amihan: Metro Manila may register something between 22 to 31 degrees Celsius, 20 to 29 degrees in Tuguegarao, 21 to 32 degrees in Laoag, 13 to 24 degrees in Baguio, 21 to 29 degrees in Tagaytay, and 24 to 31 degrees in Puerto Princesa. Legazpi may also see lower temperatures and rains on Thursday due to the possi

## Importing contents to JSON file

In [9]:
import json

json_dump = json.dumps(news_json, indent = 4) 

with open("news.json", "w") as outfile:
    outfile.write(json_dump)